# Quadro geral para visualização no GeoSampa

Esse aqui é o NoteBook Para Planejamento da Visualização dos dados no GeoSampa


In [1]:
import pandas as pd
import glob

# Abrindo o ano de 2020
df = pd.read_csv('download/dados-do-sp156---1o-tri-2020.csv',index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer'])

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace(',', '.'), errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'].str.replace(',', '.'), errors='coerce')

In [3]:
df.dtypes

Data de abertura         datetime64[ns]
Canal                            object
Tema                             object
Assunto                          object
Serviço                          object
Logradouro                       object
Número                          float64
CEP                              object
Subprefeitura                    object
Distrito                         object
Setor                           float64
Quadra                          float64
Latitude                        float64
Longitude                       float64
Data do parecer          datetime64[ns]
Status da solicitação            object
Órgão                            object
dtype: object

## Suprimindo da visualização serviços pouco solicitados 

Para melhorar a navagação seria interessante ter menos serviços dessa forma podemos criar um grupo grande com outros serviços

In [4]:
limite = 200

In [5]:
%pprint

Pretty printing has been turned OFF


In [6]:
df[~df.Distrito.isna()].groupby(['Serviço']).filter(lambda x: len(x) > limite).groupby(['Tema','Serviço'])['Canal'].count().to_csv(f'./resultados/hierarquia-de-visualizacao-limite-{limite}.csv')

In [7]:
df = df.drop(df.groupby(['Serviço']).filter(lambda x: len(x) <= limite).index)

## Pontos

No caso dos pontos podemos gerar um arquivo sem discretização, ou seja, cada registro conterá seu ponto.

Então para todos os registros vamos criar uma geometria de pontos pela Lat e Long

In [8]:
import geopandas as gpd

In [9]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [10]:
gdf.crs = 'epsg:4989'
gdf = gdf.to_crs('epsg:31983')

## Transformando Setor e Quadra em pontos

Alguns registros de atendimento estão com informações de quadras ou setores fiscais e portanto deveriam ser representado pela feição do polígono

Porém, para evitarmos múltiplas feições ou uma subtotalização transformar aqueles registros em um ponto no centroide da feição, uma vez que conhecemos as feições de Setor e Quadra. Como são casos menos frequentes podemos proporcionar uma melhor análise dessa maneira.

Alias, poderíamos criar um ponto aleatório na feição para evitar a sobreposição, o que também pode ser uma estratégia.

In [11]:
zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_setorfiscal.zip!SIRGAS_SHP_setorfiscal/SIRGAS_SHP_setorfiscal.shp"
df_setores = gpd.read_file(zipfile)

zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp"
df_quadras = gpd.read_file(zipfile)

In [12]:
df_setor = gdf[gdf.Latitude.isna() & ~gdf.Setor.isna() & gdf.Quadra.isna()]
df_quadra = gdf[gdf.Latitude.isna() & ~gdf.Setor.isna() & ~gdf.Quadra.isna()]

In [13]:
df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
df_quadra['qd_fiscal'] = df_quadra.Quadra.astype(int).apply(lambda x: '{:03d}'.format(x))

<ipython-input-13-95edc678f8ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
<ipython-input-13-95edc678f8ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
<ipython-input-13-95edc678f8ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [14]:
df_setor.st_codigo = df_setor.st_codigo.astype('str')
df_quadra.qd_setor = df_quadra.qd_setor.astype('str')
df_quadra.qd_fiscal = df_quadra.qd_fiscal.astype('str')

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
# REmovendo setores duplicados
df_setores = df_setores.drop(df_setores[df_setores.duplicated(['st_codigo'], keep=False)].index)

In [16]:
df_setores.geometry = df_setores.geometry.centroid

In [17]:
df_setor['id'] = df_setor.index

<ipython-input-17-dbee181b6a7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_setor['id'] = df_setor.index


In [18]:
setor = df_setor.merge(df_setores[['geometry', 'st_codigo']], on=['st_codigo'], suffixes=('', '_y'), how='left')

In [19]:
setor = setor.set_index('id')

In [20]:
# Atualizando os centroind no DataFrame
gdf.loc[(gdf.Latitude.isna() & ~gdf.Setor.isna() & gdf.Quadra.isna()), 'geometry'] = setor.geometry_y

In [23]:
# REmovendo as quadras duplicadas
quadras = df_quadras.drop(df_quadras[df_quadras.duplicated(['qd_setor', 'qd_fiscal', 'qd_subqua'], keep=False)].index)

In [24]:
quadras = quadras.drop(quadras[quadras.qd_subqua != '001'].index)

In [25]:
quadras.geometry = quadras.geometry.centroid

In [26]:
df_quadra['id'] = df_quadra.index

<ipython-input-26-b7af52435b04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quadra['id'] = df_quadra.index


In [27]:
quadra = df_quadra.merge(quadras[['geometry', 'qd_setor', 'qd_fiscal']], on=['qd_setor', 'qd_fiscal'], suffixes=('', '_y'), how='left')

In [28]:
quadra = quadra.set_index('id')

In [29]:
# Atualizando os centroind da quadra no DataFrame
gdf.loc[gdf.Latitude.isna() & ~gdf.Setor.isna() & ~gdf.Quadra.isna(), 'geometry'] = quadra.geometry_y

## Totalizando serviços por distrito

Quando o atendimento gera apenas uma informação do distrito uma estratégia é totalizar os atendimentos por cada distrito.

In [30]:
df_distritos = gpd.read_file('arquivos/SIRGAS_SHP_distrito_polygon.shp')

In [31]:
df_distritos.crs = 'epsg:31983'

In [32]:
df_distrito = df[~df.Distrito.isna()]

In [33]:
len(df_distrito.Tema.unique())

12

In [34]:
len(df_distrito.Serviço.unique())

110

In [35]:
# pd.get_dummies(df[['Distrito', 'Serviço']][~df['Distrito'].isna()], columns=['Serviço']).groupby(['Distrito']).sum()

In [36]:
for t in df_distrito.Tema.unique():
    print(f"{t}: ({len(df_distrito[(df_distrito.Tema == t)])})")
    for s in df_distrito[(df_distrito.Tema == t)].Serviço.unique():
        print(f"  {s}: ({len(df_distrito[(df_distrito.Serviço == s)])})")

Rua e bairro: (91652)
  Reclamação de poluição sonora - PSIU: (4804)
  Tapa-buraco: (38036)
  Drenagem de água da chuva - Bueiros e bocas de lobo: (2033)
  Manutenção de guias, sarjetas e sarjetões: (1721)
  Fiscalização de ambulantes: (1073)
  Execução de serviços em árvore localizada em área pública: (15745)
  Roçada em praças e canteiro central: (5138)
  Manutenção de calçadas em áreas públicas e praças: (469)
  Capinação em guias e sarjetas: (5791)
  Limpeza de bueiros, boca de lobo e poços de visita: (5911)
  Invasão de áreas municipais: (611)
  Obstáculo dificultando acessibilidade: (1038)
  Denúncia de calçada particular irregular, danificada ou inexistente: (873)
  Denunciar falta de limpeza de terrenos e imóveis particulares: (3101)
  Reforma e revitalização de praças: (501)
  Denunciar poda ou remoção não autorizada de árvore: (626)
  Pintura de guias e postes: (271)
  Recuperar galerias de águas de chuvas (pluviais): (765)
  Denunciar despejo irregular de água usada ou esgot

## Gerando o arquivo das camadas

Agora vamos gerar um arquivo GeoPackage com um padrão de nomes assim:

* Tema - Geometria - Serviço

Isso tanto para os pontos quanto para as totalizações por distrito

In [37]:
for t in df_distrito.Tema.unique():
    print(f"{t}: ({len(df_distrito[(df_distrito.Tema == t)])})")
    for s in df_distrito[(df_distrito.Tema == t)].Serviço.unique():
        print(f"  {s}: ({len(df_distrito[(df_distrito.Serviço == s)])})")
        c = (df_distrito[(df_distrito.Serviço == s)][['Distrito','Data de abertura']]
             .groupby('Distrito')
             .count()
             .rename(columns={'Data de abertura':'total_atendimentos'}))
        r = df_distritos.merge(c, left_on='ds_nome', right_on='Distrito')
        if len(r) > 1:
            r.to_file(f'resultados/Visualizador-156-2020.gpkg', layer=f'{t} - Polígono - {s}', driver='GPKG')

Rua e bairro: (91652)
  Reclamação de poluição sonora - PSIU: (4804)
  Tapa-buraco: (38036)
  Drenagem de água da chuva - Bueiros e bocas de lobo: (2033)
  Manutenção de guias, sarjetas e sarjetões: (1721)
  Fiscalização de ambulantes: (1073)
  Execução de serviços em árvore localizada em área pública: (15745)
  Roçada em praças e canteiro central: (5138)
  Manutenção de calçadas em áreas públicas e praças: (469)
  Capinação em guias e sarjetas: (5791)
  Limpeza de bueiros, boca de lobo e poços de visita: (5911)
  Invasão de áreas municipais: (611)
  Obstáculo dificultando acessibilidade: (1038)
  Denúncia de calçada particular irregular, danificada ou inexistente: (873)
  Denunciar falta de limpeza de terrenos e imóveis particulares: (3101)
  Reforma e revitalização de praças: (501)
  Denunciar poda ou remoção não autorizada de árvore: (626)
  Pintura de guias e postes: (271)
  Recuperar galerias de águas de chuvas (pluviais): (765)
  Denunciar despejo irregular de água usada ou esgot

In [38]:
# Exportanto os pontos

In [39]:
pontos = gdf[gdf.geometry.is_valid]

In [40]:
for t in pontos.Tema.unique():
    print(f"{t}: ({len(pontos[(pontos.Tema == t)])})")
    for s in pontos[(pontos.Tema == t)].Serviço.unique():
        print(f"  {s}: ({len(pontos[(pontos.Serviço == s) & (pontos.Tema == t)])})")
        r = pontos[(pontos.Serviço == s) & (pontos.Tema == t)]
        if len(r) > 1:
            r.to_file(f'resultados/Visualizador-156-2020.gpkg', layer=f'{t} - Ponto - {s}', driver='GPKG')

Rua e bairro: (51234)
  Tapa-buraco: (38129)
  Capinação em guias e sarjetas: (5806)
  Limpeza de bueiros, boca de lobo e poços de visita: (5920)
  Pintura de guias e postes: (271)
  Tapa-buraco em faixa exclusiva ou corredor de ônibus: (1108)
Trânsito e Transporte: (8301)
  Ônibus - Reclamação de intervalo excessivo da linha: (4724)
  Fiscalização de veículo estacionado em local proibido: (1295)
  Ônibus - Denúncia de direção inadequada ou perigosa de veículo: (934)
  Manutenção de placas de trânsito: (742)
  Manutenção de sinalização horizontal (pintura de solo): (606)
Lixo e limpeza: (34640)
  Remoção de grandes objetos em vias públicas: (15629)
  Reclamações sobre coleta de lixo domiciliar: (1756)
  Instalação e remoção de lixeira: (2312)
  Reclamação de falta de varrição: (3703)
  Remoção de entulho em via pública: (7997)
  Lavagem especial de locais públicos: (578)
  Cata Bagulho - Reclamação de não retirada de material: (1053)
  Limpeza da via pública após enchentes ou eventos: 

## TODO Trantando pontos fora dos limites do município de São Paulo

Existem casos em que os pontos estão localizados fora dos limites do município de São Paulo e para finalidade de estudo vamos quantifica-los e discriminá-los para estudos posteriores.

In [41]:
# msp = geopandas.read_file('arquivos/SP.shp')

In [42]:
# gdf_sp = geopandas.sjoin(gdf, msp, how='left', op='within')

In [43]:
# gdf_sp[gdf_sp.municipio.isna()].to_file("resultados/fora_dos_limites.gpkg", layer='fora_dos_limites', driver="GPKG")